In [2]:
import pandas as pd
from pyecharts import Pie, Bar, Line
import folium
import json

## 고령층 감염경로

In [3]:
# 코로나 확진자 데이터 불러오기
patient = pd.read_csv('PatientInfo.csv')
aged_pat = patient[(patient['age'] == '60s')|(patient['age'] == '70s')|
                (patient['age'] == '80s')|(patient['age'] == '90s')][['province','age','infection_case']]
aged_inf = pd.DataFrame(aged_pat['infection_case'].value_counts())

In [4]:
# 감염 경로를 한글로 변경
# 확진자 수가 3명 미만인 곳은 '기타'로 표시
aged_inf.rename(index={'contact with patient':'확진자와의 접촉', 'etc':'기타', 'overseas inflow':'해외 유입',
                      'Bonghwa Pureun Nursing Home':'요양원', 'Shincheonji Church':'신천지 교회',
                      "Eunpyeong St. Mary's Hospital":'병원', 'Gyeongsan Seorin Nursing Home':'요양원',
                      'Gyeongsan Jeil Silver Town':'실버 타운', 'Guro-gu Call Center':'구로구 콜센터',
                      'Gyeongsan Cham Joeun Community Center':'커뮤니티 센터', 'Cheongdo Daenam Hospital':'병원',
                      'Onchun Church':'교회', 'Dongan Church':'교회', 'Geochang Church':'교회',
                      'gym facility in Cheonan':'기타', 'Seongdong-gu APT':'기타', 'Pilgrimage to Israel':'기타',
                      'Changnyeong Coin Karaoke':'기타'}, inplace=True)
aged_inf2 = aged_inf.groupby(aged_inf.index).sum().sort_values(by='infection_case', ascending=False)

In [5]:
attr = aged_inf2.index
v1 = aged_inf2['infection_case']

pie = Pie("고령층 확진자 감염 경로", "그래프에 마우스를 올려놓으면 장소별 고령층 확진자 수를 나타냅니다",
          width=950, height=700)
pie.add("",attr, v1, center=[45,50], radius=[30,75], is_label_show=True, label_text_size=15,
        legend_orient='vertical', legend_pos='right', legend_text_size=14)
pie

앞서 연령대별 코로나 현황을 파악한 결과, 비교적 적은 확진자 수에 비해 치사율이 눈에 띄게 높은 고령층에 초점을 두고 코로나 감염 경로를 분석해보았습니다.

여기서 주목해야 할 점은 주로 알려진 감염 경로 이외에도 요양원이나 실버 타운 등에서 적지 않은 수가 감염이 되었다는 사실인데요, 이에 따라 교회나 병원뿐만 아니라 고령층이 집단으로 생활하는 시설에는 더욱 철저한 생활 방역이 필요하다는 점을 알 수 있습니다.

## 음압병상 및 격리병상 현황

In [6]:
# 전국 음압/격리병상 데이터 불러오기
# 출처 : 보건복지부 및 질병관리본부
sickbed = pd.read_csv('음압_격리병상.csv', header=1)
sickbed_df = sickbed[['city','sickbed_total']].rename(columns={'city':'province'})

In [7]:
# 코로나 확진자 데이터 지역 이름 한글로 변경
aged_pat.loc[aged_pat['province'] == 'Seoul', "province"] = "서울특별시"
aged_pat.loc[aged_pat['province'] == 'Gyeonggi-do', "province"] = '경기도'
aged_pat.loc[aged_pat['province'] == 'Gangwon-do', "province"] = '강원도'
aged_pat.loc[aged_pat['province'] == 'Chungcheongbuk-do', "province"] = '충청북도'
aged_pat.loc[aged_pat['province'] == 'Chungcheongnam-do', "province"] = '충청남도'
aged_pat.loc[aged_pat['province'] == 'Jeollabuk-do', "province"] = '전라북도'
aged_pat.loc[aged_pat['province'] == 'Jeollanam-do', "province"] = '전라남도'
aged_pat.loc[aged_pat['province'] == 'Gyeongsangbuk-do', "province"] = '경상북도'
aged_pat.loc[aged_pat['province'] == 'Gyeongsangnam-do', "province"] = '경상남도'
aged_pat.loc[aged_pat['province'] == 'Jeju-do', "province"] = '제주특별자치도'
aged_pat.loc[aged_pat['province'] == 'Ulsan', "province"] = '울산광역시'
aged_pat.loc[aged_pat['province'] == 'Daejeon', "province"] = '대전광역시'
aged_pat.loc[aged_pat['province'] == 'Gwangju', "province"] = '광주광역시'
aged_pat.loc[aged_pat['province'] == 'Incheon', "province"] = '인천광역시'
aged_pat.loc[aged_pat['province'] == 'Daegu', "province"] = '대구광역시'
aged_pat.loc[aged_pat['province'] == 'Busan', "province"] = '부산광역시'

In [8]:
aged_pat2 = pd.DataFrame(aged_pat[['province','age']].groupby('province').count())
aged_pat2.drop(['Sejong'], axis=0, inplace=True)
aged_pat2 = aged_pat2.sort_values(by=['province']).reset_index(inplace=False)

In [9]:
aged_pat3 = pd.merge(aged_pat2, sickbed_df, on='province', how='outer')

In [10]:
attr1 = ['강원','경기','경남','경북','광주','대구','대전','부산','서울','울산','인천','전남','전북','충남','충북','제주']
v1 = aged_pat3['sickbed_total']

bar = Bar("지역별 음압병상 및 격리병상 현황", "그래프에 마우스를 올려놓으면 음압/격리병상 수를 나타냅니다",
         width=950, height=500)
bar.add("음압/격리병상 개수", attr1, v1, mark_point=['max','min'], mark_line=['average'],
          is_label_show=True, label_pos='inside', label_color=['#4F788D'], legend_pos='right')
bar

코로나와 같은 감염병 환자의 경우 바이러스가 병실 밖으로 확산되는 것을 방지하기 위해 음압병실에서 치료를 받는 것이 원칙이지만, 병실이 부족한 상황에서는 일반 격리병실을 이용하게 됩니다.

따라서 의료인프라의 현황을 파악하고자 전국 음압병상 데이터와 국가지정 격리병상 데이터를 활용해보았습니다. 그 결과 서울에는 평균을 훨씬 뛰어 넘는 435개의 음압/격리병상이 있는 반면, 경기도와 경상남도, 부산광역시를 제외한 모든 지역이 평균에 미치지 못하는 것으로 나타났습니다.

## 고령층 확진자 대비  음압/격리병상 수용력

In [11]:
# 지역별 고령층 확진자 대비 병상 수용력 칼럼 추가
aged_pat3['capacity'] = round((aged_pat3['sickbed_total']/aged_pat3['age']), 1)

# 고령층 확진자가 없는 제주도 행 생략
aged_pat4 = aged_pat3.drop(15, axis=0, inplace=False)

In [12]:
attr2 = ['강원','경기','경남','경북','광주','대구','대전','부산','서울','울산','인천','전남','전북','충남','충북']
v2 = aged_pat4['capacity']

bar = Bar("고령층 확진자 수 대비 음압/격리병상 수용력", "그래프에 마우스를 올려놓으면 수용률을 나타냅니다",
         width=950, height=500)
bar.add("음압/격리병상 수용률 (배)", attr2, v2, mark_point=['max','min'], mark_line=['average'],
       is_label_show=True, label_pos='inside', label_color=['#4C82B4'], legend_pos='right', yaxis_formatter='배')
bar

이번 코로나 사태에 따른 고령층 확진자 수에 비추어 보았을 때, 의료인프라 부족 현상은 더욱 심각합니다.

고령층 확진자만을 대상으로 비교한 것임에도 불구하고, 광주광역시와 전라남도를 제외한 모든 지역에서 음압/격리병상의 수용력이 저조한 모습을 보이고 있습니다. 특히, 경기도나 경상북도의 경우에는 고령층 확진자조차 제대로 수용하지 못했다는 사실을 알 수 있습니다.

## 지역별 예상 고령층 확진자 대비 음압/격리병상 수용력

In [13]:
# 전체 고령층 확진자 수 구하기
confirmed = pd.read_csv('TimeAge.csv')
confirmed = confirmed[confirmed['date']=='2020-04-30'][['age', 'confirmed']].reset_index(drop=True)
aged_confirmed = confirmed[(confirmed['age'] == '60s')|(confirmed['age'] == '70s')|
                          (confirmed['age'] == '80s')]['confirmed'].sum()

In [14]:
# 전체 인구 수 구하기 (2020년 인구 데이터 불러오기)
# 출처 : 서울시빅데이터
population = pd.read_csv('2020년_인구.txt', sep=',')
population.drop(['Unnamed: 0'], axis=1, inplace=True)
total_population = population['population'].sum()

In [15]:
# 고령층 확진률 구하기 (전체 인구 수 대비 전체 고령층 확진자 수)
confirm_rate = aged_confirmed/total_population
confirm_rate

4.903247186057881e-05

In [16]:
# 지역별 인구 수 구하기
population2 = population[['city','population']].rename(columns={'city':'province'})
pop_df = pd.DataFrame(population2.groupby('province').sum())
pop_df.drop(['세종특별자치시'], axis=0, inplace=True)
pop_df.reset_index(inplace=True)

# 전국 병상 + 지역별 인구 수 데이터
df1 = pd.merge(sickbed_df, pop_df, on='province')
df1.sort_values(by=['province'], inplace=True)
df1 = df1.reset_index(drop=True)

In [17]:
# 지역별 예상 고령층 확진자 수 구하기 (고령층 확진률 * 지역별 인구 수)
df1['pre_confirm'] = round(confirm_rate*df1['population'], 0)

# 지역별 예상 고령층 확진자 대비 음압/격리 병상 수용력 칼럼 추가
df1['capacity2'] = round((df1['sickbed_total']/df1['pre_confirm']), 2)
df1.head()

,province,sickbed_total,population,pre_confirm,capacity2
0,강원도,52,1538577,75.0,0.69
1,경기도,163,13288975,652.0,0.25
2,경상남도,98,3355341,165.0,0.59
3,경상북도,67,2653418,130.0,0.52
4,광주광역시,36,1455705,71.0,0.51


In [18]:
attr3 = ['강원','경기','경남','경북','광주','대구','대전','부산','서울','울산','인천','전남','전북','제주','충남','충북']
v3 = df1['capacity2']

bar = Bar("예상 고령층 확진자 대비 음압/격리병상 수용력", "그래프에 마우스를 올려놓으면 수용률을 나타냅니다",
         width=950, height=500)
bar.add("음압/격리병상 수용률 (배)", attr3, v3, mark_point=['max','min'], mark_line=['average'],
       is_label_show=True, label_pos='inside', label_color=['#BD8964'], legend_pos='right', yaxis_formatter='배')
bar

대구광역시나 경상북도가 아닌 다른 지역에서 집단 감염이 발생할 경우를 가정하여, 지역별 인구 수에 고령층 코로나 확진률을 곱함으로써 지역별로 예상되는 고령층 확진자 수를 구해보았습니다. 그에 대한 음압/격리병상의 수용력은 위의 그래프와 같습니다.

만일 경기도나 인천광역시, 전라북도, 대전광역시 등의 지역에서 집단 감염이 발생했다면 더 큰 문제가 될 수 있었던 상황이기 때문에, 지역별 음압/격리병상 수를 더욱 확대할 필요가 있을 것으로 보입니다.

## 지역별 예상 고령층 확진자 대비 필요한 음압/격리병상 수

In [19]:
# capacity2가 평균 미만인 지역별 필요한 병상 수 계산
capacity2_mean = df1['capacity2'].mean()

In [20]:
df1['needed_bed'] = round(capacity2_mean*df1['pre_confirm'] - df1['sickbed_total'], 0)
df2 = df1[df1['needed_bed'] > 0][['province','needed_bed']]
df2.reset_index(drop=True, inplace=True)
df2

,province,needed_bed
0,경기도,201.0
1,경상북도,6.0
2,광주광역시,4.0
3,대전광역시,13.0
4,울산광역시,3.0
5,인천광역시,27.0
6,전라북도,16.0
7,충청남도,14.0


In [21]:
# 행정구역 데이터 불러오기
# 출처 : 공공데이터포털
with open('korea2.json', 'r',encoding='utf-8') as f:
    data = json.load(f)

In [22]:
map1 = folium.Map(location=[37.565,126.986],zoom_start=7,tiles='OpenStreetMap')
map1.choropleth(geo_data = data, data=df2,
               columns=['province','needed_bed'],
               key_on='feature.properties.CTP_KOR_NM',
               fill_color='YlGn',
               legend_name = 'Needed Sickbed')
map1

C:\Users\Playdaya\anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


그렇다면, 지역별로 몇 개의 음압/격리병상이 필요한 것일까요? 위의 지도는 예상 고령층 확진자 대비 음압/격리병상 수용력이 평균 미만인 지역을 대상으로 필요한 병상 수를 계산하여 지도로 나타낸 결과입니다.

색이 진할수록 필요한 음압/격리병상의 수가 많다는 것을 의미하며, 병상을 추가로 설치하지 않아도 무방한 지역은 검은 색으로 표시되고 있습니다.

이에 따라 음압/격리병상의 수용력을 평균 수준으로 향상시키기 위해서는 경기도가 201개, 인천광역시가 27개, 전라북도가 16개 정도의 음압/격리병상을 필요로 한다는 사실을 알 수 있습니다.